In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import re

In [192]:
zones = ["", "China","emerg","Europe","Global","India","Japan","Rest"]

def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x

def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_unleveraged_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_leverage/beta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x

def get_wacc(zone=""):
    path = "data_damodaran/discount_rate_estimation/wacc_damodaran/wacc" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_spread_classifier(size=5e9, financial=False):
    path = "data_damodaran/capital_structure/spread_classifiers/ratings.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Start here Ratings sheet"]
    moody = process_df(x.iloc[16:32,:4])
    moody_financefirms = process_df(x.iloc[16:,5:9])
    moody_smallfirms = process_df(x.iloc[35:,:4])
    
    if financial == True:
        return moody_financefirms
    else:
        if size >= 5e9:
            return moody
        else: 
            return moody_smallfirms
        
        
def get_10y_tbills():
    path = "data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["T. Bond yield & return"]
    x = x.iloc[5:,:]
    x = process_df(x)
    return x

def get_moody_spread(x, size=5e9, financial=False):
    for i in range(len(get_spread_classifier(size,financial=financial))):
        if x <= get_spread_classifier(size,financial=financial).iloc[i,1]:
            return get_spread_classifier(size,financial=financial).iloc[i,-1]
        else:
            continue
    return get_spread_classifier(size,financial=financial).iloc[-1,-1]


def get_risk_premium():
    path = "data_damodaran/discount_rate_estimation/risk_premium/ctrypremJuly23.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["ERPs by country"]
    x = x.iloc[5:163,:9]
    x = process_df(x)
    x = x.set_index("Country")
    return x


get_risk_premium()

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Africa,Moody's rating,Rating-based Default Spread,Total Equity Risk Premium,Country Risk Premium,"Sovereign CDS, net of US",Total Equity Risk Premium2,Country Risk Premium3
Country,,,,,,,,
Abu Dhabi,Middle East,Aa2,0.005281,0.05749,0.00749,0.0027,0.053829,0.003829
Albania,Eastern Europe & Russia,B1,0.048154,0.118298,0.068298,NaN,NaN,NaN
Andorra (Principality of),Western Europe,Baa2,0.020368,0.078888,0.028888,NaN,NaN,NaN
Angola,Africa,B3,0.069528,0.148613,0.098613,0.1004,0.192399,0.142399
Argentina,Central and South America,Ca,0.128369,0.232068,0.182068,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
Uruguay,Central and South America,Baa2,0.020368,0.078888,0.028888,0.0052,0.057375,0.007375
Uzbekistan,Eastern Europe & Russia,Ba3,0.038473,0.104567,0.054567,NaN,NaN,NaN
Venezuela,Central and South America,C,0.175,0.298205,0.248205,0.1114,0.208,0.158


In [134]:
x = yf.Ticker("NVDA")
x.info.get("industry")

sector = "Semiconductor"

### Capital Structure

In [149]:
E = x.balancesheet.loc["Stockholders Equity",:].iloc[0]
D = x.balancesheet.loc["Total Liabilities Net Minority Interest",:].iloc[0]
print("Debt:",D,"Equity:",E)

Debt: 19081000000.0 Equity: 22101000000.0


### free-risk rate

In [214]:
rf = get_10y_tbills().iloc[-1,1]
rf

0.0388

### Rd Damodaran Estimation

In [215]:
rd = get_wacc().loc[sector,"Cost of Debt"]

In [216]:
interest = x.incomestmt.loc["Interest Expense",:]
ebit = x.incomestmt.loc["EBIT",:]
coverage_ratio = ebit / interest

coverage_ratio.apply(get_moody_spread)

spread = get_moody_spread(coverage_ratio.iloc[-1])
spread

rd = rf + spread
rd 

0.046200000000000005

### Re Damodaran Estimation

In [217]:
re = get_wacc().loc[sector,"Cost of Equity"]

## re estimated

In [218]:
tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]
beta_u = get_unleveraged_betas().loc[sector,"Unlevered beta"]
beta = beta_u * (1 + (1-tax_rate)*D/E)
prm = get_risk_premium().loc[x.info["country"],"Total Equity Risk Premium"]

re = rf + beta * prm
re

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


0.1703842750054868

## WACC

In [219]:
def wacc(rd,re,t,E,D):
    wacc = rd*(1-t)*(D/(E+D)) + re*(E/(D+E))
    return wacc



In [220]:
wacc(rd,re,tax_rate,E,D)

0.11065939332769223